In [1]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

import time

pd.set_option('display.max_columns', 50)

In [2]:
years = list(range(2024,2021, -1))
years

[2024, 2023, 2022]

In [3]:
connect_timeout = 6
read_timeout = 100

def stats(link, year):
     
    data = requests.get(link, timeout=(connect_timeout, read_timeout))
    soup = BeautifulSoup(data.text)
    
    stats_table = pd.read_html(data.text, match="Regular season Table")[0]
    
    standard_stats = pd.read_html(data.text, match = "Squad Standard Stats")[0]
    standard_stats.columns = standard_stats.columns.droplevel()
    
    goalkeeping_stats = pd.read_html(data.text, match="Squad Goalkeeping")[0]
    goalkeeping_stats.columns = goalkeeping_stats.columns.droplevel()
    
    shooting_stats = pd.read_html(data.text, match="Squad Shooting")[0]
    shooting_stats.columns = shooting_stats.columns.droplevel()
    
    stats_standard = stats_table.merge(standard_stats[["Squad", "Poss", "Gls", "Ast"]], on="Squad", how="left")
    stats_standard_gk = stats_standard.merge(goalkeeping_stats[["Squad", "CS%"]], on="Squad", how="left")
    all_stats = stats_standard_gk.merge(shooting_stats[["Squad", "SoT/90", "G/SoT", "SoT%"]], on="Squad", how="left")
        
    all_stats["Season"] = year
    
    return all_stats
    

In [4]:
overall_stats = []

In [5]:
def concate():
    
    all_stats_2024 = stats("https://fbref.com/en/comps/9/Premier-League-Stats", 2024)
    all_stats_2024 = all_stats_2024.drop(["Last 5"], axis=1)
    all_stats_2023 = stats("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats", 2023)
    all_stats_2022 = stats("https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats", 2022)
    all_stats_2021 = stats("https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats", 2021)
    all_stats_2020 = stats("https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats", 2020)
    all_stats_2019 = stats("https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats", 2019)
    
    overall_stats.append(all_stats_2024)
    overall_stats.append(all_stats_2023)
    overall_stats.append(all_stats_2022)
    overall_stats.append(all_stats_2021)
    overall_stats.append(all_stats_2020)
    overall_stats.append(all_stats_2019)
    
    overall_stats_df = pd.concat(overall_stats)
    return overall_stats_df
    

In [6]:
overall_stats_df = concate()

In [7]:
overall_stats_df.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes,Poss,Gls,Gls,Ast,Ast,CS%,SoT/90,G/SoT,SoT%,Season
0,1,Liverpool,23,15,6,2,52,22,30,51,2.22,48.9,28.1,20.9,0.91,53065,Mohamed Salah - 14,Alisson,NaN,59.6,48,2.09,38,1.65,34.8,6.26,0.31,33.9,2024
1,2,Manchester City,22,15,4,3,54,25,29,49,2.23,43.6,21.6,21.9,1.00,53182,Erling Haaland - 14,Ederson,NaN,64.9,53,2.41,40,1.82,22.7,6.77,0.33,40.1,2024
2,3,Arsenal,23,15,4,4,47,22,25,49,2.13,44.3,17.5,26.8,1.16,60196,Bukayo Saka - 8,David Raya,NaN,60.1,46,2.00,31,1.35,34.8,5.00,0.35,30.7,2024
3,4,Aston Villa,23,14,4,5,49,30,19,46,2.00,40.8,30.7,10.1,0.44,41558,Ollie Watkins - 11,Emiliano Martínez,NaN,55.8,46,2.00,34,1.48,26.1,5.22,0.35,36.3,2024
4,5,Tottenham,23,13,5,5,49,35,14,44,1.91,40.3,39.7,0.6,0.03,61586,Son Heung-min - 12,Guglielmo Vicario,NaN,59.6,46,2.00,39,1.70,21.7,5.78,0.34,36.6,2024


In [8]:
overall_stats_df.shape

(120, 29)

In [9]:
overall_stats_df.to_csv("../datasets/overall_stats2.csv", index=False)